# Packages

In [1]:
# Load Packages 
import pandas as pd
import numpy as np
import math
import os
import random
from itertools import combinations
from tqdm import tqdm

from statsmodels.tsa.tsatools import lagmat
import statsmodels.api as sm
from statsmodels.tsa.ar_model import AutoReg

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn import random_projection
from sklearn.tree import DecisionTreeRegressor

from dimod import BinaryQuadraticModel
from dwave.samplers import SimulatedAnnealingSampler

# import pyreadr
# 
# from dwave.preprocessing import roof_duality
# from dimod import ExactSolver
from dwave.samplers import SteepestDescentSolver
# from dwave.samplers import TabuSampler
# from dwave.samplers import TreeDecompositionSolver
# import neal

# Set Path

In [17]:
# Set Path
path  =  os.getcwd() # os.path.dirname(os.getcwd()) #r'/Users/slehmann/Library/CloudStorage/Dropbox/QUBO'

# Pre-Process Data

In [4]:
# Make Fred-MD-Data stationary
def transform_tcode(data):
    
    # Get Transformation-Code
    tcode = data[0]
    
    # Get Data
    data = data[1:]

    if tcode == 1:
        output = data
    elif tcode == 2:
        output = data - np.roll(data, 1)
    elif tcode == 3:
        output = (data - np.roll(data, 1)) - (np.roll(data, 1) - np.roll(data, 2))
    elif tcode == 4:
        output = np.log(data)
    elif tcode == 5:
        output = np.log(data) - np.roll(np.log(data), 1)
    elif tcode == 6:
        output = (np.log(data) - np.roll(np.log(data), 1)) - (np.roll(np.log(data), 1) - np.roll(np.log(data), 2))
    else:
        output = (data / np.roll(data, 1) - 1) - (np.roll(data, 1) / np.roll(data, 2) - 1)

    return np.concatenate(([tcode], output))


In [5]:
# Function to create Lags
def create_lags(y, X, mlags):
    
    # Create all lags from zero to maxlag
    data_lagged  =  lagmat(y, maxlag = mlags, use_pandas = True)
    
    # Add to Feature Matrix
    X = data_lagged.join(X)
    
    # Return
    return X

In [6]:
# Function to Pre-Process Data
def prepro(y, X, t):
    
    # Train Data
    y_train  =  y.iloc[:t, :].copy()
    X_train  =  X.iloc[:t, :].copy()

    # Predict Data
    y_pred  =  y.iloc[[t]].copy()
    X_pred  =  X.iloc[[t]].copy()
    
    # Standardize Data
    scaler  =  StandardScaler()   
    X_train[X_train.columns] =  scaler.fit_transform(X_train[X_train.columns])
    X_pred[X_pred.columns]   =  scaler.transform(X_pred[X_pred.columns])
    
    # Add Constant
    X_train =  sm.add_constant(X_train)
    X_pred  =  sm.add_constant(X_pred, has_constant = 'add')
    
    return y_train, X_train, y_pred, X_pred

# (Complete) Subset Forecasts

In [7]:
# Function to return array of all subsets of length k
def complete_sub(arr, k):
    
    ### Elements are treated as unique based on their position, not on their value.
    ### So if the input elements are unique, there will be no repeated values in each combination.
    
    # Get all subsets of size k
    subset  =  list(combinations(arr, k)) 
    
    # Return 
    return subset 

In [8]:
# Function to calculate number of models
def n_models(K, k):
    return math.factorial(K) / (math.factorial(k) * math.factorial(K-k))

In [9]:
# Function to randomly select n_max items from array
def random_select(arr, n_max, random_state):
    
    # Set random state
    random.seed(random_state)
    
    # Set upper Boundary
    upper_bound  =  len(arr) if len(arr) < n_max else n_max
    
    # Randomly select items without repetition
    rand_arr  =  random.sample(arr, k = upper_bound)
    
    # Return 
    return rand_arr

In [10]:
# Function to produce Subset Regression Forecasts
def ssf(y_train, X_train, X_pred, feature, mlags):
    
    # Subset Feature Space (incl. constant)
    X_train_subset = X_train.iloc[:, list(range(0, mlags+1)) + list(feature)]
    X_pred_subset  = X_pred.iloc[:, list(range(0, mlags+1)) + list(feature)]
    
    # Fit Model
    model = sm.OLS(y_train, X_train_subset)
    regr = model.fit()
    
    # Predict
    pred = regr.predict(X_pred_subset)
    
    return pred

# Compressed Regressions

In [11]:
# Compressed Regression (Gaussian random projection)
def cr_reg(y_train, X_train, X_pred, n_comp, mlags, ran_st):
    
    # Set up Random-Projection-Matrix
    transformer = random_projection.GaussianRandomProjection(n_components = n_comp, random_state = ran_st)
    
    # Transform
    tmp_train  =  transformer.fit_transform(X_train.iloc[:, (mlags+1):])
    tmp_pred   =  transformer.fit_transform(X_pred.iloc[:, (mlags+1):])

    # Convert to DataFrame
    tmp_train  =  pd.DataFrame(tmp_train, index = X_train.index)
    tmp_pred   =  pd.DataFrame(tmp_pred,  index = X_pred.index)

    # Add Constant + Lags
    comp_matrix_train =  pd.concat([X_train.iloc[:, :(mlags+1)], tmp_train], axis = 1)
    comp_matrix_pred  =  pd.concat([X_pred.iloc[:, :(mlags+1)],  tmp_pred],  axis = 1)

    # Fit Model
    model  =  sm.OLS(y_train, comp_matrix_train)
    regr   =  model.fit()
    
    # Predict
    pred = regr.predict(comp_matrix_pred)
    
    return pred

# Decision Trees

In [13]:
# Decision Tree Regression
def dt_reg(y_train, X_train, X_pred, ran_st):
    
    # Set up Regressor Object 
    model  =  DecisionTreeRegressor(criterion = "squared_error",
                                    max_depth = 20,
                                    splitter  = "random",
                                    random_state = ran_st)
    
    # Fit model
    model.fit(X_train, y_train)
    
    # Predict
    pred = model.predict(X_pred)

    # Return Prediction
    return pred

# AR-Model

In [14]:
# Autoregressive Model
def ar_mod(y_train, lags):

    # Fit AR-Model
    model = AutoReg(y_train, lags=lags).fit()
    
    # Prediction
    pred = model.forecast(steps=1)

    # Return Prediction
    return pred

# Best Selection of Forecasts

In [15]:
# Best Selection of Forecasts
def bssf(Y_train, X_train, X_pred, alpha, n_sub, n_times):
    
    # Adapt X-Matrix
    X_train  =  X_train / n_sub
    
    # Generate Q-Matrix
    ivec      =  np.mat(np.ones(X_train.shape[1])).transpose()
    aux_mat   =  np.array(Y_train.transpose() @ X_train + alpha * n_sub)
    diag_mat  =  np.diag(aux_mat[0])
    Q         =  - 2 * diag_mat + X_train.transpose() @ X_train + alpha * ivec @ ivec.transpose()

    # Initialize BQM
    bqm  =  BinaryQuadraticModel('BINARY')
    bqm  =  bqm.from_qubo(Q)
    
    # Solve
    #solver     =  SimulatedAnnealingSampler()
    solver     =  SteepestDescentSolver()
    #solver     =  TabuSampler()
    #solver     =  TreeDecompositionSolver()

    sampleset  =  solver.sample(bqm, num_reads = n_times)
    solution   =  list(sampleset.first[0].values())
    
    # Prediction 
    pred       = solution @ X_pred
    
    # Return 
    return(pred, solution)

# Application

In [18]:
# Load Data
x_dataset  =  pd.read_csv(path + r'/fred_md_202306.csv')

# Drop Variables with too many missing values
x_dataset  =  x_dataset.drop(["PERMIT", "PERMITNE", "PERMITMW", "PERMITS",
                              "PERMITW", "ACOGNO", "ANDENOx", "CP3Mx",
                              "COMPAPFFx", "TWEXAFEGSMTHx", "UMCSENTx", "VIXCLSx"],
                              axis=1)

# Transform remaining Columns
x_dataset.iloc[:, 1:]  =  x_dataset.iloc[:,1:].apply(lambda x: transform_tcode(x))

# Drop First Row with Transformation-Code
x_dataset  =  x_dataset.iloc[1:,:]

# Lag Data
x_dataset.iloc[:,1:]  =  x_dataset.iloc[:,1:].shift(1)

# Convert Date
x_dataset['sasdate']  =  pd.to_datetime(x_dataset['sasdate'])

# Filter Data
x_dataset  =  x_dataset[(x_dataset['sasdate'] >= '1959-04-01') & (x_dataset['sasdate'] <= '2023-03-01')]

# Resetting the index
x_dataset.set_index('sasdate', inplace=True)
x_dataset.index = x_dataset.index.to_period('M')

In [19]:
# Load Data
y_dataset  =  pd.read_csv(path + r'/fred_md_202306.csv')

# Select and Rename Variables
y_dataset  =  y_dataset.loc[:, ["sasdate", "CPIAUCSL", "INDPRO", "UNRATE"]]
y_dataset  =  y_dataset.rename(columns={'CPIAUCSL': 'CPIAUCSL_h1', 'INDPRO': 'INDPRO_h1', 'UNRATE': 'UNRATE_h1'})

# Transform Variables
y_dataset[["CPIAUCSL_h1"]]  =  y_dataset[["CPIAUCSL_h1"]].apply(lambda x: 4 * 100 * (np.log(x) - np.roll(np.log(x), 1)))
y_dataset[["INDPRO_h1"]]    =  y_dataset[["INDPRO_h1"]].apply(lambda x: 4 * 100 * (np.log(x) - np.roll(np.log(x), 1)))
y_dataset[["UNRATE_h1"]]    =  y_dataset[["UNRATE_h1"]]

# Drop First Row with Transformation-Code
y_dataset  =  y_dataset.iloc[1:,:]

# Convert Date
y_dataset['sasdate']  =  pd.to_datetime(y_dataset['sasdate'])

# Filter Data
y_dataset  =  y_dataset[(y_dataset['sasdate'] >= '1959-04-01') & (y_dataset['sasdate'] <= '2023-03-01')]

# Resetting the index
y_dataset.set_index('sasdate', inplace=True)
y_dataset.index = y_dataset.index.to_period('M')

In [43]:
###### Set Seed ######
#random.seed(123)

###### Data ######
# Set Target Variable
y  =  y_dataset.loc[:, ["CPIAUCSL_h1"]]
X  =  x_dataset.drop("CPIAUCSL", axis = 1)

# Number of AR-Terms to include
mlags =  4

# Create Lags
X  =  create_lags(y, X, mlags)

# Drop Missing Values
y  =  y[mlags:]
X  =  X[mlags:]

###### Parameter Subset Forecasts
# Subset Lengths
k_range = [1, 2, 3] # [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

# Upper Bound for Subset-Sizes
n_max = 200  # 20000

# Number of Subset-Forecasts
n_sub  =  int(sum([min(n_models((X.shape[1]-mlags), k), n_max) for k in k_range]))

###### Parameter Compressed Regressions ######
# Number of Components for Compressed Regression
cr_range  =  [1, 2, 3] # [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100]

# Number of runs for each random projection
rep_range  =  range(0, 5) # 10000

# Number of Compressed-Regression-Forecasts
n_cr  =  len(cr_range) * len(rep_range)

###### Parameter Decision Tree ######
dt_range  =  range(0, 5) # 300000

# Number of Decision Trees
n_dt  =  len(dt_range)

###### Parameter BSSF ######
alpha       =  1
n_times     =  100
bssf_range  =  range(1, 3)
n_bssf      =  len(bssf_range)

### General Parameter ######
# Initial Training-Period
init       =  12 * 20

# Total Length
total =  len(y) 

# Set up Matrices for Results
cand_forecasts  =  np.full((total, (n_sub + n_cr + n_dt)), np.nan)
benchmark       =  np.full(total, np.nan)
bssf_opt        =  np.full(total, np.nan)
sse_bssf        =  np.zeros(n_bssf)

###### Start ######
# Loop over Time
for t in tqdm(range(init, total)):
        
    # Pre-Process Data
    y_train, X_train, y_pred, X_pred = prepro(y, X, t)
    
    ### Benchmark: AR(4)
    pred          =  ar_mod(y_train, lags = mlags)
    benchmark[t]  =  pred.iloc[0]
    
    ### Subset Forecasts
    # Set up List to store Subset-Forecasts
    preds_ss  =  np.full(n_sub, np.nan)
    idx_sub   =  0
    
    # Loop over Subset Size 
    for k in k_range:
    
        # Get all possible Subset of length k
        col_idx   =  list(range(mlags+1, X_train.shape[1]))
        subs_idx  =  complete_sub(col_idx, k)

        # Randomly select n_upper Subsets
        feature_set  =  random_select(subs_idx, n_max, random_state = 123)

        # Loop over Subsets
        for feature in feature_set:

            # Compute Subset-Regression-Forecast
            pred  =  ssf(y_train, X_train, X_pred, feature, mlags)
            preds_ss[idx_sub] = pred.iloc[0]
            idx_sub += 1
            
    ### Compressed Regressions
    # Set up List to store Compressed-Regression-Forecasts
    preds_cr   = np.full(n_cr, np.nan)
    idx_cr     = 0
    
    # Loop over number of Components
    for n_comp in cr_range:

        # Loop over n repetitions
        for r in rep_range:
        
            # Compute Compressed-Regression-Forecasts
            pred  =  cr_reg(y_train, X_train, X_pred, n_comp, mlags, r)
            preds_cr[idx_cr] = pred.iloc[0]
            idx_cr += 1
            
    ### Decision Tree Regressions
    # Set up Matrix to store Decision-Tree-Forecasts
    preds_dt   = np.full(n_dt, np.nan)
    
    # Loop over number of Components
    for idx_dt, r in enumerate(dt_range):
        
        # Compute Decision-Tree-Forecasts
        pred  =  dt_reg(y_train, X_train, X_pred, r)
        preds_dt[idx_dt] = pred[0]

    # Append Results
    cand_forecasts[t][:n_sub]             =  preds_ss 
    cand_forecasts[t][n_sub:(n_sub+n_cr)] =  preds_cr
    cand_forecasts[t][(n_sub+n_cr):]      =  preds_dt

    ### Best Selection of Forecast
    if t > init:
    
        # Set up Matrix to store Forecasts
        bssf_forecasts  =  np.full(n_bssf, np.nan)
    
        # Get "best" Subset-Size until now (lowest Sum of Squared Errors)
        s_opt  =  np.argmin(sse_bssf)
    
        # Loop over Subset Sizes
        for idx_bssf, s in enumerate(bssf_range):
    
            # Compute Best-Subset-Selection-of-Forecasts
            pred  =  bssf(y_train[init:], cand_forecasts[init:t], cand_forecasts[t], alpha, s, n_times)
            bssf_forecasts[idx_bssf]  =  pred[0]
    
            # Compute Sum of Squared Errors
            sse_bssf[idx_bssf] =  sse_bssf[idx_bssf] + (y_pred.iloc[0,0] - pred[0]) ** 2
    
        # Select Forecast 
        bssf_opt[t] =  bssf_forecasts[s_opt]
    
# Convert Results to DataFrame
cand_forecasts  =  pd.DataFrame(cand_forecasts, index = y.index)
benchmark       =  pd.DataFrame(benchmark, index = y.index)
bssf_opt        =  pd.DataFrame(bssf_opt,  index = y.index)

# Cut off initial Training-Period
sub_y              =  y.iloc[init:]
sub_cand_forecasts =  cand_forecasts.iloc[init:]
sub_benchmark      =  benchmark.iloc[init:]
sub_bssf_opt       =  bssf_opt.iloc[init:]

# OOS-Period
oos_start  =  "1990-01"
oos_end    =  "2022-12" 
oos_y             =  sub_y.loc[oos_start:oos_end]
oos_cand_forecast =  sub_cand_forecasts.loc[oos_start:oos_end]
oos_benchmark     =  sub_benchmark.loc[oos_start:oos_end]
oos_bssf_opt      =  sub_bssf_opt.loc[oos_start:oos_end]

# Evaluation
np.sum((oos_y.iloc[:,0] - oos_bssf_opt.iloc[:,0]) ** 2) / np.sum((oos_y.iloc[:,0] - oos_benchmark.iloc[:,0]) ** 2)

100%|██████████| 524/524 [08:21<00:00,  1.04it/s]


0.9948131648115085

-------------

------------------

# Best Selection of Forecasts

In [ ]:
# Best Selection of Forecasts
def bssf(Y_train, X_train, X_pred, alpha, n_sub, n_times):
    
    # Adapt X-Matrix
    X_train  =  X_train / n_sub
    
    # Generate Q-Matrix
    ivec      =  np.mat(np.ones(X_train.shape[1])).transpose()
    aux_mat   =  np.array(Y_train.transpose() @ X_train + alpha * n_sub)
    diag_mat  =  np.diag(aux_mat[0])
    Q         =  - 2 * diag_mat + X_train.transpose() @ X_train + alpha * ivec @ ivec.transpose()

    # Initialize BQM
    bqm  =  BinaryQuadraticModel('BINARY')
    bqm  =  bqm.from_qubo(Q)
    
    # Solve
    solver     =  SimulatedAnnealingSampler()
    #solver     =  SteepestDescentSolver()
    #solver     =  TabuSampler()
    #solver     =  TreeDecompositionSolver()

    sampleset  =  solver.sample(bqm, num_reads = n_times)
    solution   =  list(sampleset.first[0].values())
    
    # Prediction 
    pred       = solution @ X_pred
    
    # Return 
    return(pred, solution)

In [ ]:
# Time-Series-Cross-Validation to Tune Parameter k
def cv_ts(Y, X, splits, k_seq, alpha, n_times):

    # Set up TS-Split
    tscv    =  TimeSeriesSplit(n_splits = splits)
    
    # Set up Result-Matrix
    cv_res  =  np.zeros((splits, len(k_seq)))

    # Loop over Train-Test-Splits
    for i, (train_index, test_index) in enumerate(tscv.split(X)):

        # Split Data
        X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
        Y_train, Y_test = Y.iloc[train_index],   Y.iloc[test_index]

        # Cross-Validation k
        for k in k_seq:

            # Selected Predictors
            solution    =  bssf(Y_train, X_train, X_pred, alpha, k, n_times)[1]

            # Prediction
            prediction  =  solution @ X_test.transpose()

            # MSE
            cv_res[i, k-1]  =  np.mean((Y_test.squeeze() - prediction) ** 2)

            # Select k with smalltest average MSE
            k  =  cv_res.mean(axis=0).argmin() + 1
            
    # Return 
    return(k)

In [ ]:
# Candidate Forecasts
cand_forecasts  =  results

# Target Variable
target_var  =  y_dataset.loc[:, ["CPIAUCSL_h1"]]

# Get Dates
dates  =  cand_forecasts.index.values

# Match Time Frames
target_var  =  target_var.loc[dates]

In [ ]:
# Set Parameter
alpha    =  1
n_mods   =  cand_forecasts.shape[1]

# Set Vector & Matrices
X     =  cand_forecasts.copy()
Y     =  target_var.copy()

# Set Time Parameter
init    =  int(12 * 5)
final   =  len(Y)

# Set up Empty Array
predictions  =  np.zeros(final)
predictions.fill(np.nan)

# Loop
for t in tqdm(range(init, final)):
    
    # Train Data
    X_train  =  X.iloc[:t, ]
    Y_train  =  Y.iloc[:t, ]
    
    # Prediction Data
    X_pred   =  X.iloc[t, ]
    
    # Cross-Validation
    splits   =  5
    n_times  =  500
    k_seq    =  [1, 2, 3, 4, 5]
    k_opt    =  cv_ts(Y_train, X_train, splits, k_seq, alpha, n_times)
    
    # Prediction 
    n_times  =  500
    predictions[t]  =  bssf(Y_train, X_train, X_pred, alpha, k_opt, n_times)[0]

# Convert to Pandas Series
predictions  =  pd.Series(predictions, name = "qubo", index = Y.index).to_frame()

#### Load Data

In [ ]:
# Load Target Variable
target_var      =  pyreadr.read_r(path + '/Data/Results/Target_Var/target_var.RDS')[None]

# Load all Candidate Forecasts
cand_forecasts  =  pd.DataFrame()
files           =  os.scandir(path + '/Data/Results/Candidate_Forecasts')

# Loop
for file in files:
    if (file.path.endswith(".RDS")):
        aux  =  pyreadr.read_r(file)[None]
        cand_forecasts  =  pd.concat([cand_forecasts, aux], axis = 1)
        
# Drop Na
cand_forecasts  =  cand_forecasts.dropna()

# Get Dates
dates           =  cand_forecasts.index.values

# Match Time Frames
target_var      =  target_var.loc[dates]

# Dimensions
print(cand_forecasts.shape)
print(target_var.shape)

### Set up Q-Matrix

In [ ]:
# Best Selection of Forecasts
def bssf(Y_train, X_train, X_pred, alpha, n_sub, n_times):
    
    # Adapt X-Matrix
    X_train  =  X_train / n_sub
    
    # Generate Q-Matrix
    ivec      =  np.mat(np.ones(X_train.shape[1])).transpose()
    aux_mat   =  np.array(Y_train.transpose() @ X_train + alpha * n_sub)
    diag_mat  =  np.diag(aux_mat[0])
    Q         =  - 2 * diag_mat + X_train.transpose() @ X_train + alpha * ivec @ ivec.transpose()

    # Initialize BQM
    bqm  =  BinaryQuadraticModel('BINARY')
    bqm  =  bqm.from_qubo(Q)
    
    # Solve
    solver     =  SimulatedAnnealingSampler()
    #solver     =  SteepestDescentSolver()
    #solver     =  TabuSampler()
    #solver     =  TreeDecompositionSolver()

    sampleset  =  solver.sample(bqm, num_reads = n_times)
    solution   =  list(sampleset.first[0].values())
    
    # Prediction 
    pred       = solution @ X_pred
    
    # Return 
    return(pred, solution)

In [ ]:
# Time-Series-Cross-Validation to Tune Parameter k
def cv_ts(Y, X, splits, k_seq, alpha, n_times):

    # Set up TS-Split
    tscv    =  TimeSeriesSplit(n_splits = splits)
    
    # Set up Result-Matrix
    cv_res  =  np.zeros((splits, len(k_seq)))

    # Loop over Train-Test-Splits
    for i, (train_index, test_index) in enumerate(tscv.split(X)):

        # Split Data
        X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
        Y_train, Y_test = Y.iloc[train_index],   Y.iloc[test_index]

        # Cross-Validation k
        for k in k_seq:

            # Selected Predictors
            solution    =  bssf(Y_train, X_train, X_pred, alpha, k, n_times)[1]

            # Prediction
            prediction  =  solution @ X_test.transpose()

            # MSE
            cv_res[i, k-1]  =  np.mean((Y_test.squeeze() - prediction) ** 2)

            # Select k with smalltest average MSE
            k  =  cv_res.mean(axis=0).argmin() + 1
            
    # Return 
    return(k)

In [ ]:
# Set Parameter
alpha    =  1
n_mods   =  cand_forecasts.shape[1]

# Set Vector & Matrices
X     =  cand_forecasts.copy()
Y     =  target_var.copy()

# Set Time Parameter
init    =  int(12 * 5)
final   =  len(Y)

# Set up Empty Array
predictions  =  np.zeros(final)
predictions.fill(np.nan)

# Loop
for t in range(init, final):
    
    # Train Data
    X_train  =  X.iloc[:t, ]
    Y_train  =  Y.iloc[:t, ]
    
    # Prediction Data
    X_pred   =  X.iloc[t, ]
    
    # Cross-Validation
    splits   =  5
    n_times  =  500
    k_seq    =  [1, 2, 3, 4, 5]
    k        =  cv_ts(Y_train, X_train, splits, k_seq, alpha, n_times)
    
    # Prediction 
    n_times  =  500
    predictions[t]  =  bssf(Y_train, X_train, X_pred, alpha, k, n_times)[0]

# Convert to Pandas Series
predictions  =  pd.Series(predictions, name = "qubo", index = Y.index).to_frame()

### Evaluation

In [ ]:
# Define Eval-Function (OOS-R2)
def oos_r2(observation, prediction, benchmark):
    
    # Squared Error Model
    se1  =  (observation - prediction) ** 2
    
    # Squared Error Benchmark
    se2  =  (observation - benchmark) ** 2
    
    # Out-of-Sample R2
    oos_r2  =  (1 - sum(se1) / sum(se2)) * 100
    
    # Return 
    return(oos_r2)

In [ ]:
# Set OOS-Period
eval_start  =  "1974-12-01"
eval_end    =  "2020-12-01"

# Keep only OOS-Period
oos_target_var      =  target_var.loc[eval_start:eval_end].squeeze()
oos_benchmark       =  cand_forecasts[eval_start:eval_end]["pred_hist_mean"]
oos_bssf            =  predictions.loc[eval_start:eval_end].squeeze()
oos_cand_forecasts  =  cand_forecasts[eval_start:eval_end]

# Evaluate Best Subset Selection of Forecasts
oos_r2(oos_target_var, oos_bssf, oos_benchmark)

In [ ]:
# Evaluate Candidate Models
eval_cand_mods  =  oos_cand_forecasts.apply(lambda x: oos_r2(oos_target_var, x, oos_benchmark), axis = 0)

In [ ]:
eval_cand_mods.sort_values(ascending = False).to_frame("OOS-R2").head(n = 50)

In [ ]:
# Set OOS-Period
eval_start  =  "1974-12-01"
eval_end    =  "2020-12-01"

# Keep only OOS-Period
oos_target_var      =  target_var.loc[eval_start:eval_end].squeeze()
oos_benchmark       =  cand_forecasts[eval_start:eval_end]["pred_hist_mean"]
oos_cand_forecasts  =  cand_forecasts[eval_start:eval_end]

# Evaluate Candidate Models
eval_cand_mods  =  oos_cand_forecasts.apply(lambda x: oos_r2(oos_target_var, x, oos_benchmark), axis = 0)

# Show Results
print(eval_cand_mods.filter(like = "XGB"))
print(eval_cand_mods.filter(like = "GBM"))
print(eval_cand_mods.filter(like = "pcr"))
print(eval_cand_mods.filter(like = "glm"), n =)
#eval_cand_mods.sort_values(ascending = False).to_frame("OOS-R2").head(n = 50)

In [ ]:
# Initialize BQM
qubo  =  { (i,j) : Q.iloc[i,j] for i in range(0, 11) for j in range(0, 11) }
bqm   =  BinaryQuadraticModel('BINARY')
bqm   =  bqm.from_qubo(Q)

# Initialize BQM
bqm = BinaryQuadraticModel('BINARY')

# Add Linear Coefficients
for i in range(0,11):
    lin_coef  =  Q.iloc[i,i]
    bqm.add_linear(i, lin_coef)
    
# Add Quadratic Coefficients
for i in range(0,11):
    for j in range(0,11):
        if i != j:
            quad_coef  =  Q.iloc[i,j]
            bqm.add_quadratic(i, j, quad_coef)